In [2]:
# 2D CNN
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

# 2D CNN 모델 정의
class Simple2DCNN(nn.Module):
    def __init__(self):
        super(Simple2DCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 160 * 160, 512)  # 640x640 크기에 맞게 조정
        self.fc2 = nn.Linear(512, 2)  # 예시: 클래스가 2개 (변화 감지, 비변화)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 64 * 160 * 160)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def extract_objects(results, confidence_threshold):
    objects = []
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = r
        if score > confidence_threshold:
            objects.append({
                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                'score': score,
                'class_id': int(class_id)
            })
    return objects

def compare_objects(objects1, objects2, iou_threshold):
    added = []
    removed = []
    
    matched1 = set()
    matched2 = set()
    
    for i, obj1 in enumerate(objects1):
        for j, obj2 in enumerate(objects2):
            if i in matched1 or j in matched2:
                continue
            
            iou = calculate_iou(obj1['bbox'], obj2['bbox'])
            if iou > iou_threshold:
                matched1.add(i)
                matched2.add(j)
                break
        
        if i not in matched1:
            removed.append(obj1)
    
    for j, obj2 in enumerate(objects2):
        if j not in matched2:
            added.append(obj2)
    
    return added, removed

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou
    pass    

def visualize_changes(image1, image2, added, removed):
    result_image = np.hstack((image1, image2))
    height, width = image1.shape[:2]
    
    for obj in added:
        x1, y1, x2, y2 = obj['bbox']
        cv2.rectangle(result_image, (x1 + width, y1), (x2 + width, y2), (0, 255, 0), 2)
        cv2.putText(result_image, 'Added', (x1 + width, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    for obj in removed:
        x1, y1, x2, y2 = obj['bbox']
        cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(result_image, 'Removed', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    cv2.imshow('Changes Detected', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    pass

def analyze_results(objects1, objects2, added, removed):
    print(f"Image 1: {len(objects1)} objects detected")
    print(f"Image 2: {len(objects2)} objects detected")
    print(f"Added objects: {len(added)}")
    print(f"Removed objects: {len(removed)}")
    
    class_counts1 = {}
    class_counts2 = {}
    
    for obj in objects1:
        class_counts1[obj['class_id']] = class_counts1.get(obj['class_id'], 0) + 1
    
    for obj in objects2:
        class_counts2[obj['class_id']] = class_counts2.get(obj['class_id'], 0) + 1
    
    print("\nClass distribution in Image 1:")
    for class_id, count in class_counts1.items():
        print(f"Class {class_id}: {count}")
    
    print("\nClass distribution in Image 2:")
    for class_id, count in class_counts2.items():
        print(f"Class {class_id}: {count}")


# 변화 감지 함수
def detect_changes_2d(image1_path, image2_path, model_path, cnn_model, confidence_threshold=0.5, iou_threshold=0.5):
    # YOLO 모델 로드
    model = YOLO(model_path)

    # 이미지 로드
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # YOLO를 사용하여 객체 탐지
    results1 = model(image1)[0]
    results2 = model(image2)[0]

    # 객체 추출
    objects1 = extract_objects(results1, confidence_threshold)
    objects2 = extract_objects(results2, confidence_threshold)

    # 객체 비교 (기존 방식 유지)
    added, removed = compare_objects(objects1, objects2, iou_threshold)

    # 이미지를 2D CNN에 입력
    image1_2d = prepare_image_for_2d_cnn(image1)
    image2_2d = prepare_image_for_2d_cnn(image2)

    # 2D CNN을 통한 특징 추출 및 변화 감지
    features1 = cnn_model(image1_2d)
    features2 = cnn_model(image2_2d)
    
    difference = compute_difference_2d(features1, features2)

    # 결과 시각화
    visualize_changes(image1, image2, added, removed)

    # 결과 분석
    analyze_results(objects1, objects2, added, removed)

# 2D CNN에 입력할 이미지를 준비하는 함수
def prepare_image_for_2d_cnn(image):
    # 이미지를 2D CNN에 입력할 수 있도록 변환
    image = cv2.resize(image, (640, 640))
    image_2d = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
    return torch.Tensor(image_2d).unsqueeze(0)  # (B, C, H, W)

# 2D CNN을 통한 특징 비교 함수
def compute_difference_2d(features1, features2):
    # 간단한 유클리드 거리 계산
    return torch.norm(features1 - features2, p=2).item()

# 나머지 기존 함수들 유지 (extract_objects, compare_objects, calculate_iou, visualize_changes, analyze_results)

# CNN 모델 초기화
cnn_model = Simple2DCNN()

# 사용 예시
image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 1.jpg'
image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 2.jpg'
model_path = 'C:/Users/AI-LHJ/Desktop/ultralytics-main/runs/segment/train8 best/weights/best.pt'

detect_changes_2d(image1_path, image2_path, model_path, cnn_model)



0: 640x640 5 crossarms, 1 polo, 13 wires, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 49.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 crossarms, 1 polo, 15 wires, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Image 1: 17 objects detected
Image 2: 20 objects detected
Added objects: 3
Removed objects: 0

Class distribution in Image 1:
Class 2: 11
Class 0: 5
Class 1: 1

Class distribution in Image 2:
Class 2: 14
Class 0: 5
Class 1: 1


In [3]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Simple2DCNN(nn.Module):
    def __init__(self):
        super(Simple2DCNN, self).__init__()
        # 더 깊은 CNN 구조로 변경
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 80 * 80, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(torch.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(torch.relu(self.batch_norm3(self.conv3(x))))
        x = x.view(-1, 256 * 80 * 80)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

def extract_objects(results, confidence_threshold):
    objects = []
    # 신뢰도 점수에 따른 정렬 추가
    boxes = sorted(results.boxes.data.tolist(), key=lambda x: x[4], reverse=True)
    
    for r in boxes:
        x1, y1, x2, y2, score, class_id = r
        if score > confidence_threshold:
            # 바운딩 박스 정규화 추가
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            w, h = x2 - x1, y2 - y1
            if w > 0 and h > 0:  # 유효한 바운딩 박스만 추가
                objects.append({
                    'bbox': [x1, y1, x2, y2],
                    'score': score,
                    'class_id': int(class_id),
                    'area': w * h  # 면적 정보 추가
                })
    return objects

def compare_objects(objects1, objects2, iou_threshold):
    added = []
    removed = []
    matched1 = set()
    matched2 = set()
    
    # 면적 기반 정렬 추가
    objects1 = sorted(objects1, key=lambda x: x['area'], reverse=True)
    objects2 = sorted(objects2, key=lambda x: x['area'], reverse=True)
    
    for i, obj1 in enumerate(objects1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(objects2):
            if j in matched2:
                continue
                
            iou = calculate_iou(obj1['bbox'], obj2['bbox'])
            # 클래스 ID가 같은 경우에만 매칭
            if iou > best_iou and obj1['class_id'] == obj2['class_id']:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            matched1.add(i)
            matched2.add(best_match)
        else:
            removed.append(obj1)
    
    for j, obj2 in enumerate(objects2):
        if j not in matched2:
            added.append(obj2)
    
    return added, removed

def detect_changes_2d(image1_path, image2_path, model_path, cnn_model, 
                     confidence_threshold=0.45,  # 신뢰도 임계값 조정
                     iou_threshold=0.4):        # IOU 임계값 조정
    # YOLO 모델 로드 및 설정
    model = YOLO(model_path)
    model.conf = confidence_threshold  # 모델 신뢰도 임계값 설정
    model.iou = iou_threshold         # 모델 IOU 임계값 설정
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 정규화
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO 예측
    results1 = model(image1, verbose=False)[0]  # verbose=False로 불필요한 출력 제거
    results2 = model(image2, verbose=False)[0]
    
    # 객체 추출 및 비교
    objects1 = extract_objects(results1, confidence_threshold)
    objects2 = extract_objects(results2, confidence_threshold)
    
    # 변화 감지
    added, removed = compare_objects(objects1, objects2, iou_threshold)
    
    # CNN 특징 추출
    with torch.no_grad():  # 추론 모드로 변경
        image1_2d = prepare_image_for_2d_cnn(image1)
        image2_2d = prepare_image_for_2d_cnn(image2)
        features1 = cnn_model(image1_2d)
        features2 = cnn_model(image2_2d)
    
    # 결과 시각화 및 분석
    visualize_changes(image1, image2, added, removed)
    analyze_results(objects1, objects2, added, removed)
    
    return added, removed

def prepare_image_for_2d_cnn(image):
    # 이미지 정규화 추가
    image = image.astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))
    image = torch.Tensor(image).unsqueeze(0)
    return image

# 사용 예시
if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 7.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 8.jpg'
    model_path = 'C:/Users/AI-LHJ/Desktop/ultralytics-main/runs/segment/train8 best/weights/best.pt'
    
    # CNN 모델 초기화 및 평가 모드로 설정
    cnn_model = Simple2DCNN()
    cnn_model.eval()
    
    # 변화 감지 실행
    detect_changes_2d(image1_path, image2_path, model_path, cnn_model)

Image 1: 11 objects detected
Image 2: 11 objects detected
Added objects: 0
Removed objects: 0

Class distribution in Image 1:
Class 0: 2
Class 2: 8
Class 1: 1

Class distribution in Image 2:
Class 0: 2
Class 2: 8
Class 1: 1


In [34]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Simple2DCNN(nn.Module):
    def __init__(self, input_channels=3):
        super(Simple2DCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 160 * 160, 512)
        self.fc2 = nn.Linear(512, 2)  # 변화 있음/없음 분류

    def forward(self, x):
        x = self.pool(torch.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(torch.relu(self.batch_norm2(self.conv2(x))))
        x = x.view(-1, 128 * 160 * 160)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def create_detection_map(image_shape, detections, confidence_threshold=0.25):
    """YOLO 검출 결과를 히트맵으로 변환"""
    detection_map = np.zeros(image_shape[:2], dtype=np.float32)
    
    for det in detections.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = det
        if conf > confidence_threshold:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            detection_map[y1:y2, x1:x2] = conf
    
    return detection_map

def detect_changes(image1_path, image2_path, yolo_model_path, cnn_model=None):
    # YOLO 모델 로드
    yolo = YOLO(yolo_model_path)
    
    # 이미지 로드 및 크기 조정
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출
    results1 = yolo(image1)[0]
    results2 = yolo(image2)[0]
    
    # 검출 결과를 히트맵으로 변환
    detection_map1 = create_detection_map(image1.shape, results1)
    detection_map2 = create_detection_map(image2.shape, results2)
    
    # 검출 맵을 3채널로 확장 (CNN 입력용)
    detection_maps = np.stack([
        detection_map1,
        detection_map2,
        np.abs(detection_map1 - detection_map2)
    ], axis=2)
    
    # 결과 시각화
    visualize_results(image1, image2, detection_map1, detection_map2, results1, results2)
    
    # CNN 모델이 제공된 경우 변화 분류 수행
    if cnn_model is not None:
        with torch.no_grad():
            # 입력 데이터 준비
            input_tensor = torch.FloatTensor(detection_maps).permute(2, 0, 1).unsqueeze(0)
            
            # 변화 예측
            prediction = cnn_model(input_tensor)
            probability = torch.softmax(prediction, dim=1)
            change_prob = probability[0][1].item()
            
            print(f"Change probability: {change_prob:.2%}")
            return change_prob > 0.5
    
    return None

def visualize_results(image1, image2, map1, map2, yolo_results1, yolo_results2):
    # YOLO 검출 결과 시각화
    det_image1 = image1.copy()
    det_image2 = image2.copy()
    
    # YOLO 바운딩 박스 그리기
    for result, img in [(yolo_results1, det_image1), (yolo_results2, det_image2)]:
        for box in result.boxes.data.tolist():
            x1, y1, x2, y2, conf, cls = box
            if conf > 0.25:  # 신뢰도 임계값
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f'{conf:.2f}', (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # 히트맵 시각화
    map1_colored = cv2.applyColorMap((map1 * 255).astype(np.uint8), cv2.COLORMAP_JET)
    map2_colored = cv2.applyColorMap((map2 * 255).astype(np.uint8), cv2.COLORMAP_JET)
    
    # 결과 합치기
    top_row = np.hstack((det_image1, det_image2))
    bottom_row = np.hstack((map1_colored, map2_colored))
    combined = np.vstack((top_row, bottom_row))
    
    # 창 크기 조정
    scale = 0.7
    display_size = (int(combined.shape[1] * scale), int(combined.shape[0] * scale))
    combined = cv2.resize(combined, display_size)
    
    cv2.imshow('Detection Results', combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 5.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 6.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/ultralytics-main/runs/segment/train8 best/weights/best.pt'
    
    # 2D CNN 모델 초기화 (선택적)
    cnn_model = Simple2DCNN(input_channels=3)
    
    # 변화 감지 실행
    result = detect_changes(image1_path, image2_path, yolo_model_path, cnn_model)
    
    if result is not None:
        print(f"Change detected: {result}")


0: 640x640 4 crossarms, 1 polo, 15 wires, 18.3ms
Speed: 2.0ms preprocess, 18.3ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 crossarms, 1 polo, 19 wires, 20.1ms
Speed: 2.6ms preprocess, 20.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Change probability: 36.46%
Change detected: False


In [36]:
import cv2
import numpy as np
from ultralytics import YOLO

def calculate_iou(box1, box2):
    """두 바운딩 박스 간의 IoU 계산"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection + 1e-6)
    return iou

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25, iou_threshold=0.5):
    # YOLO 모델 로드
    model = YOLO(yolo_model_path)
    
    # 이미지 로드
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출
    results1 = model(image1)[0]
    results2 = model(image2)[0]
    
    # 검출 결과 추출
    boxes1 = []
    boxes2 = []
    
    for r in results1.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes1.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    for r in results2.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes2.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    # 객체 매칭 및 변화 감지
    disappeared = []  # 사라진 객체
    appeared = []    # 새로 나타난 객체
    
    # 같은 객체 매칭
    for i, obj1 in enumerate(boxes1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(boxes2):
            if obj2['matched'] or obj1['class'] != obj2['class']:
                continue
                
            iou = calculate_iou(obj1['box'], obj2['box'])
            if iou > best_iou:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            boxes1[i]['matched'] = True
            boxes2[best_match]['matched'] = True
        else:
            disappeared.append(obj1)
    
    # 새로 나타난 객체 찾기
    for obj2 in boxes2:
        if not obj2['matched']:
            appeared.append(obj2)
    
    # 결과 시각화
    def create_visualization(image1, image2, disappeared, appeared):
        # 원본 이미지 복사
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체 표시 (빨간색)
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img1, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(vis_img1, f'Disappeared (conf: {obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체 표시 (초록색)
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis_img2, f'Appeared (conf: {obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 히트맵 생성
        heatmap1 = np.zeros(image1.shape[:2], dtype=np.float32)
        heatmap2 = np.zeros(image2.shape[:2], dtype=np.float32)
        
        # 사라진 객체 히트맵
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            heatmap1[y1:y2, x1:x2] = obj['conf']
        
        # 새로 나타난 객체 히트맵
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            heatmap2[y1:y2, x1:x2] = obj['conf']
        
        # 히트맵 컬러 변환
        heatmap1_colored = cv2.applyColorMap((heatmap1 * 255).astype(np.uint8), cv2.COLORMAP_JET)
        heatmap2_colored = cv2.applyColorMap((heatmap2 * 255).astype(np.uint8), cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, heatmap1_colored, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, heatmap2_colored, alpha, 0)
        
        # 최종 결과 이미지 생성
        result = np.hstack((overlay1, overlay2))
        
        return result, heatmap1_colored, heatmap2_colored
    
    # 시각화 실행
    result_image, hmap1, hmap2 = create_visualization(image1, image2, disappeared, appeared)
    
    # 결과 출력
    cv2.imshow('Object Changes', result_image)
    
    # 분석 결과 출력
    print("\nChange Detection Results:")
    print(f"Total objects in first image: {len(boxes1)}")
    print(f"Total objects in second image: {len(boxes2)}")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

# 실행 코드
if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 3.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 4.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/ultralytics-main/runs/segment/train8 best/weights/best.pt'
    
    disappeared, appeared = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25,  # 신뢰도 임계값
        iou_threshold=0.5     # IoU 임계값
    )


0: 640x640 2 crossarms, 1 polo, 15 wires, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 crossarms, 1 polo, 18 wires, 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Total objects in first image: 18
Total objects in second image: 21
Disappeared objects: 0
Appeared objects: 3

Appeared objects details:
Class: 2, Confidence: 0.94
Class: 2, Confidence: 0.89
Class: 2, Confidence: 0.46
